## Imports

In [2]:
from ultralytics import YOLO

## Load model 

In [5]:
# Load models
v8n_person = YOLO("train_runs/yolov8n_person_100epochs/detect/train/weights/best.pt")
v8n_mask = YOLO("train_runs/yolov8n_mask_100epochs/train/weights/best.pt")
v11n_person = YOLO("train_runs/yolo11n_person_100epochs/train/weights/best.pt")
v11n_mask = YOLO("train_runs/yolo11n_mask_100epochs/train/weights/best.pt")

# Create model list
model_list = [v8n_person, v8n_mask, v11n_person, v11n_mask]

## Test if model was loaded correctly

In [15]:
for i in range(3):
    results = v8n_person.predict(f"data_sets/image_data/mask_person_test/{i}.jpg", conf=0.5)

    # Show results
    results[0].show()

FileNotFoundError: data_sets/image_data/mask_person_test/0.jpg does not exist